## Load Data

## Markets Insider Scraping

### code is not run because it is not automated

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as ur
import pandas as pd
from requests import get

In [ ]:
res = []
date_lst = []
#change page range
or page_num in range(1, 50):
    #change ticker
    link = f'https://markets.businessinsider.com/news/aapl?p={page_num}'
    resp = get(link)
    with open('temp.html', 'wb') as fOut:
        fOut.write(resp.content)
    
    with open('temp.html','r',encoding='utf-8') as html_file:
        content = html_file.read()
        soup = BeautifulSoup(content, 'lxml')
    
    sources = soup.find_all('span', attrs = {'class':"latest-news__source"})
    #len(sources)
        
    links = soup.find_all('a', attrs = {'class':"news-link"})
    counter= 0
    for page in links:
        temp_str = ""
        #err403 = False
        try:
            source = sources[counter].text
        
            #print(source)
        
            #if source  == "Premium" or source == "Seeking Alpha":
                #counter+=1
                #continue
            url = ur.urlopen("https://markets.businessinsider.com"+page.get('href'))
            #print(url)
            content = url.read()
            article = BeautifulSoup(content, 'lxml')
        
    
            #handle differently based on source

            if source in ["BBC","Belfast Telegraph", "EN, Chinapost", "Financial Times", "Market Watch", "MarketWatch",
                         "MediaPost.com", "Moodys","MotleyFool", "Newsfile", "The Sydney Morning Herald", "USA Today", 
                          "Zacks", "fin24", "New York Times"]:
                temp_str = str(article.find_all("div", attrs = {'class':'col-xs-12 news-content no-padding'}))
                idx_start = temp_str.rfind("\r\n")
                temp_str = temp_str[idx_start+2:]
                idx_end =temp_str.find("<")
                temp_str = temp_str[:idx_end]
                #res.append(temp_str)
            
            elif source in ["Benzinga","Business Insider", "InvestorPlace", "RTTNews"]:
                text_list = article.find_all("p")[3:]
                #iterate
                for tag in text_list:
                    temp_str += tag.text
                
                #res.append(temp_str)
                #handle the case of diff tags
                #if temp_str=="":
                    #print(sources[counter])
            date = article.find_all("span", attrs = {'class':'news-post-quotetime warmGrey'})[0].text
            date_lst.append(date)   
            res.append(temp_str)
        except:
            print(f"https://markets.businessinsider.com{page.get('href')} 404 Not Found")
            message = '404 Not Found'
            res.append(message)
            date_lst.append(message)
        counter += 1
        if (len(date_lst) != len(res)):
            print("Error")
    print(page_num)
    
df = pd.DataFrame({'Date': date_lst,'Text':res})
df.to_csv('APPLE Data Part 2.csv')

## CNBC

In [ ]:
import requests

params = {
    "queryly_key": "31a35d40a9a64ab3",
    #change this part
    "query": "Facebook",
    "endindex": "0",
    "batchsize": "100",
    "callback": "",
    "showfaceted": "true",
    "timezoneoffset": "-120",
    "facetedfields": "formats",
    "facetedkey": "formats|",
    "facetedvalue":
    "!Press Release|",
    "needtoptickers": "1",
    "additionalindexes": "4cd6f71fbf22424d,937d600b0d0d4e23,3bfbe40caee7443e,626fdfcd96444f28"
}

goal = ["cn:title", "_pubDate", "cn:liveURL", "description"]


def main(url):
    with requests.Session() as req:
        allin = []
        for page, item in enumerate(range(0, 20000, 100)):
            print(f"Extracting Page# {page +1}")
            params["endindex"] = item
            r = req.get(url, params=params).json()
            for loop in r['results']:
                allin.append([loop[x] for x in goal])
        new = pd.DataFrame(
            allin, columns=["Title", "Date", "Url", "Description"])
        new.to_csv("Facebook Data CNBC.csv", index=False)
        return new


main("https://api.queryly.com/cnbc/json.aspx")

## Google Trends(Only done for Apple)

In [23]:
from pytrends.request import TrendReq
pytrends = TrendReq()
kw_list = ["Apple"] 
df = pytrends.get_historical_interest(kw_list, 
                                      year_start=2016, month_start=10, day_start=1, hour_start=0, 
                                      year_end=2021, month_end=10, day_end=1, hour_end=0, cat=0, 
                                      geo='', gprop='', sleep=120)
#df_pg.reset_index().plot(x='date', y='P&G', figsize=(120, 10), kind ='line')
df.to_csv("apple_trend.csv")

In [2]:
import tensorflow
from pandas_datareader.data import DataReader
import requests_cache
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime

### US equities(Kaggle) + MarketsInsider

In [3]:
d1 = pd.read_csv("us_equities_news_dataset.csv")
aapl_d1 = d1[d1['ticker']=='AAPL'][['release_date', 'content']].rename(columns = {'release_date':'Date', 'content':'Text'})
# facebook has no data here.
fb_d1 =d1[d1['ticker']=='FB'][['release_date', 'content']].rename(columns = {'release_date':'Date', 'content':'Text'})
pg_d1 = d1[d1['ticker']=='PG'][['release_date', 'content']].rename(columns = {'release_date':'Date', 'content':'Text'})
#import from marketsinsider
fb_d2 = pd.read_csv("Facebook Full.csv", usecols = 'Unnamed' not in x)
pg_d2 = pd.read_csv("P&G Full.csv", usecols = 'Unnamed' not in x)
aapl_d2 = pd.read_csv("APPLE Full.csv", usecols = 'Unnamed' not in x)

fb_comp = fb_d1.append(fb_d2)
pg_comp = pg_d1.append(pg_d2)
aapl_comp = aapl_d1.append(aapl_d2)

#aapl_comp.to_csv("Apple Full.csv")
#fb_comp.to_csv("Facebook Full.csv")
#pg_comp.to_csv("P&G Full.csv")

## load CNBC

In [13]:
aapl = pd.read_csv('Apple data CNBC.csv')[['Date', 'Description']].rename(columns = {'Description':'Text'})
fb = pd.read_csv('Facebook data CNBC.csv')[['Date', 'Description']].rename(columns = {'Description':'Text'})
pg = pd.read_csv('P&G data CNBC.csv')[['Date', 'Description']].rename(columns = {'Description':'Text'})
aapl.head()

,Date,Text
0,11/10/2021 5:35:20 PM,Apple announced Wednesday it's launching a bus...
1,11/10/2021 3:37:51 AM,Apple will be forced to allow app developers t...
2,11/10/2021 3:38:52 PM,German chipmaker and Apple-supplier Infineon i...
3,11/10/2021 4:06:25 PM,Tim Cook's company may have made your phone or...
4,11/10/2021 3:29:55 PM,Here are the biggest calls on Wall Street on W...


## Final Copy

In [14]:
aapl['Date'] = aapl['Date'].apply(lambda x: str(pd.to_datetime(x[:-11].strip().replace("/", "-"), format = "%m-%d-%Y"))[:10])
fb['Date'] = fb['Date'].apply(lambda x: str(pd.to_datetime(x[:-11].strip().replace("/", "-"), format = "%m-%d-%Y"))[:10])
pg['Date'] = pg['Date'].apply(lambda x: str(pd.to_datetime(x[:-11].strip().replace("/", "-"), format = "%m-%d-%Y"))[:10])
aapl.head()

,Date,Text
0,2021-11-10,Apple announced Wednesday it's launching a bus...
1,2021-11-10,Apple will be forced to allow app developers t...
2,2021-11-10,German chipmaker and Apple-supplier Infineon i...
3,2021-11-10,Tim Cook's company may have made your phone or...
4,2021-11-10,Here are the biggest calls on Wall Street on W...


In [16]:
AAPL = aapl_comp.append(aapl)
FB = fb_comp.append(fb)
PG = pg_comp.append(pg)

AAPL = AAPL.drop_duplicates(subset='Text', keep="last")
FB = FB.drop_duplicates(subset='Text', keep="last")
PG = PG.drop_duplicates(subset='Text', keep="last")

In [17]:
from flair.models import TextClassifier
from flair.data import Sentence

In [18]:
classifier = TextClassifier.load('en-sentiment')

2021-11-19 19:00:56,720 loading file C:\Users\Darius\.flair\models\sentiment-en-mix-distillbert_4.pt


### Get sentiment scores

In [19]:
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
scores = AAPL['Text'].apply(vader.polarity_scores).tolist()
AAPL = AAPL.reset_index(drop = True)
#print(AAPL)
AAPL = AAPL.join(pd.DataFrame(scores))

scores = FB['Text'].apply(vader.polarity_scores).tolist()
FB = FB.reset_index(drop = True)
#print(FB)
AAPL = AAPL.join(pd.DataFrame(scores))

scores = PG['Text'].apply(vader.polarity_scores).tolist()
PG = PG.reset_index(drop = True)
#print(PG)
PG = PG.join(pd.DataFrame(scores))

In [20]:
# we will append probability and sentiment preds later
probs = []
sentiments = []

# use regex expressions (in clean function) to clean 
for content in AAPL['text'].to_list():
    # make prediction
    sentence = flair.data.Sentence(content)
    sentiment_model.predict(sentence)
    # extract sentiment prediction
    probs.append(sentence.labels[0].score)  # numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

# add probability and sentiment predictions to tweets dataframe
AAPL['probability'] = probs
AAPL['sentiment'] = sentiments

# we will append probability and sentiment preds later
probs = []
sentiments = []

# use regex expressions (in clean function) to clean 
for content in FB['content'].to_list():
    # make prediction
    sentence = flair.data.Sentence(content)
    sentiment_model.predict(sentence)
    # extract sentiment prediction
    probs.append(sentence.labels[0].score)  # numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

# add probability and sentiment predictions to tweets dataframe
FB['probability'] = probs
FB['sentiment'] = sentiments

# we will append probability and sentiment preds later
probs = []
sentiments = []

# use regex expressions (in clean function) to clean 
for content in PG['content'].to_list():
    # make prediction
    sentence = flair.data.Sentence(content)
    sentiment_model.predict(sentence)
    # extract sentiment prediction
    probs.append(sentence.labels[0].score)  # numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

# add probability and sentiment predictions to tweets dataframe
PG['probability'] = probs
PG['sentiment'] = sentiments

In [21]:
AAPL.head()

,Date,Text,neg,neu,pos,compound,probability,sentiment
0,2021-11-10,Apple announced Wednesday it's launching a bus...,0.000,1.000,0.000,0.0000,0.621849,POSITIVE
1,2021-11-10,Apple will be forced to allow app developers t...,0.161,0.803,0.036,-0.7579,0.999783,NEGATIVE
2,2021-11-10,German chipmaker and Apple-supplier Infineon i...,0.070,0.930,0.000,-0.2846,0.999716,NEGATIVE
3,2021-11-10,Tim Cook's company may have made your phone or...,0.129,0.871,0.000,-0.8047,0.998617,NEGATIVE
4,2021-11-10,Here are the biggest calls on Wall Street on W...,0.000,0.918,0.082,0.6369,0.999696,NEGATIVE


In [22]:
AAPL.to_csv('Apple_sentiment.csv')
FB.to_csv('Facebook_sentiment.csv')
PG.to_csv('P&G_sentiment.csv')

## Import data

In [24]:
aapl_data  = pd.read_csv('Apple_sentiment.csv', usecols = lambda x:"Unnamed" not in x)
aapl_data.head()

,Date,Text,neg,neu,pos,compound,probability,sentiment
0,2021-11-10,Apple announced Wednesday it's launching a bus...,0.000,1.000,0.000,0.0000,0.621849,POSITIVE
1,2021-11-10,Apple will be forced to allow app developers t...,0.161,0.803,0.036,-0.7579,0.999783,NEGATIVE
2,2021-11-10,German chipmaker and Apple-supplier Infineon i...,0.070,0.930,0.000,-0.2846,0.999716,NEGATIVE
3,2021-11-10,Tim Cook's company may have made your phone or...,0.129,0.871,0.000,-0.8047,0.998617,NEGATIVE
4,2021-11-10,Here are the biggest calls on Wall Street on W...,0.000,0.918,0.082,0.6369,0.999696,NEGATIVE


In [25]:
fb_data = pd.read_csv('Facebook_sentiment.csv', usecols = lambda x: 'Unnamed'not in x)
fb_data.head()

,Date,Text,neg,neu,pos,compound,probability,sentiment
0,2021-11-04,As Mark Zuckerberg pursues a future in the met...,0.000,0.973,0.027,0.0772,0.970368,POSITIVE
1,2021-11-05,"Meta, formerly known as Facebook, is discussin...",0.043,0.870,0.087,0.2960,0.539886,NEGATIVE
2,2021-11-04,"LISBON, Portugal — Facebook is wrong to shut d...",0.063,0.857,0.080,0.2023,0.999805,NEGATIVE
3,2021-11-05,"Bradley Tusk, Tusk ventures founder and CEO, j...",0.000,0.872,0.128,0.4215,0.860280,POSITIVE
4,2021-11-02,Facebook on Tuesday announced it will be putti...,0.000,0.964,0.036,0.1779,0.999882,NEGATIVE


In [26]:
pg_data =pd.read_csv("P&G_sentiment.csv", usecols = lambda x: "Unnamed" not in x)
pg_data.head()

,Date,Text,neg,neu,pos,compound,probability,sentiment
0,2021-11-04,"InvestorPlace - Stock Market News, Stock Advic...",0.025,0.867,0.108,0.9987,0.923506,POSITIVE
1,2021-11-02,"InvestorPlace - Stock Market News, Stock Advic...",0.041,0.837,0.122,0.9992,0.999227,NEGATIVE
2,2021-10-29,"InvestorPlace - Stock Market News, Stock Advic...",0.028,0.871,0.101,0.9992,0.860475,POSITIVE
3,2021-10-28,Shares of InflaRx N.V. rocketed 32.2% on he...,0.021,0.819,0.160,0.9839,0.999744,NEGATIVE
4,2021-10-28,"InvestorPlace - Stock Market News, Stock Advic...",0.055,0.815,0.130,0.9976,0.999963,NEGATIVE


# Text Preprocessing for Topic Modelling

In [27]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
nltk.download('stopwords')
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models 
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from gensim import corpora, models

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Darius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
aapl_text = aapl_data.Text.values.tolist()
fb_text = fb_data.Text.values.tolist()
pg_text = pg_data.Text.values.tolist()
#preprocess texts
def sent_to_words(sentences):
    for sentence in sentences:
        #print(sentence)
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
aapl_words = list(sent_to_words(aapl_text))
fb_words = list(sent_to_words(fb_text))
pg_words = list(sent_to_words(pg_text))

In [43]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'https', 'com', 'get','go', 'use', 'www', 'fb', 'facebook','pg', 'proctor', "gamble", "apple", "aapl",'guy',
                  'would', 'should', 'might', 'may', 'could', 'can', 'cannot', 'thing', 'say' ,'still', 'make', 'people'])
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word.lower() not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [31]:
#call the functions in order
# Remove Stop Words
def clean_text(text):
  # Build the bigram and trigram models
  bigram = gensim.models.Phrases(text, min_count=5, threshold=100) # higher threshold fewer phrases.
  #trigram = gensim.models.Phrases(bigram[text], threshold=100)  

  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  #trigram_mod = gensim.models.phrases.Phraser(trigram)
  data_words_nostops = remove_stopwords(text)

  # Form Bigrams
  data_words_bigrams = [bigram_mod[doc] for doc in data_words_nostops]
  
  #data_words_trigrams = [trigram_mod[bigram_mod[doc]] for doc in texts]

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
  return data_lemmatized

aapl_cleaned = clean_text(aapl_words)
fb_cleaned = clean_text(fb_words)
pg_cleaned = clean_text(pg_words)

In [33]:
import pickle

In [ ]:
pickle_out = open("aapl_list", "wb")
pickle.dump(aapl_cleaned, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open("fb_list", "wb")
pickle.dump(fb_cleaned, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open("pg_list", "wb")
pickle.dump(pg_cleaned, pickle_out)
pickle_out.close()

In [39]:
pickle_in1 = open('aapl_list', "rb")
aapl_cleaned = pickle.load(pickle_in1)
aapl_cleaned[0]

['announce',
 'launch',
 'business',
 'tool',
 'service',
 'company',
 'employee',
 'company',
 'look',
 'new',
 'way',
 'handle',
 'technology',
 'service',
 'currently',
 'beta',
 'testing',
 'slate',
 'full',
 'release',
 'spring',
 'subscription',
 'begin']

In [40]:
pickle_in2 = open('fb_list', "rb")
fb_cleaned = pickle.load(pickle_in2)
fb_cleaned[0]

['pursue',
 'future',
 'metaverse',
 'count',
 'exist',
 'business',
 'provide',
 'capital',
 'week',
 'announce',
 'new',
 'corporate',
 'name',
 'strategy',
 'company',
 'say',
 'add',
 'number']

In [41]:
pickle_in3 = open('pg_list', "rb")
pg_cleaned = pickle.load(pickle_in3)
pg_cleaned[0]

['stock',
 'tipsoften',
 'acclaim',
 'job',
 'invest',
 'expert',
 'tough',
 'innovative',
 'management',
 'style',
 'revolutionize',
 'analyze',
 'stock',
 'buy',
 'found',
 'associate',
 'successful',
 'hedge',
 'fund',
 'asset',
 'management',
 'ace',
 'investor',
 'employ',
 'simple',
 'step',
 'formula',
 'save',
 'money',
 'create',
 'diversify',
 'portfolio',
 'understand',
 'cycle',
 'time',
 'volatility',
 'exactly',
 'kind',
 'approach',
 'need',
 'peruse',
 'bridgewater',
 'associate',
 'portfolio',
 'find',
 'many',
 'headscratcher',
 'contrast',
 'approach',
 'invest',
 'wood',
 'tend',
 'risk',
 'tolerant',
 'approach',
 'pick',
 'stock',
 'see',
 'portfolio',
 'approach',
 'successful',
 'doubt',
 'however',
 'environment',
 'predict',
 'correction',
 'take',
 'conservative',
 'route',
 'hurt',
 'top',
 'stock',
 'buy',
 'top',
 'investor',
 'vaccine',
 'business',
 'struggle',
 'keep',
 'competition',
 'example',
 'covid_vaccine',
 'sale',
 'fall',
 'short',
 'analyst',

In [42]:
aapl_cleaned = [[word for word in doc if word not in stop_words] for doc in aapl_cleaned]
fb_cleaned = [[word for word in doc if word not in stop_words] for doc in fb_cleaned]
pg_cleaned = [[word for word in doc if word not in stop_words] for doc in pg_cleaned]

# Topic Modelling

In [30]:
#def get_lda_model(text, n=30):
  dictionary_LDA = corpora.Dictionary(text)
  dictionary_LDA.filter_extremes(no_below=3)
  corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in text]

  num_topics = n
  #%time 
  lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                id2word=dictionary_LDA,passes=4, \
                                alpha = [0.01]*num_topics, \
                                eta=[0.01]*len(dictionary_LDA.keys()))
  return lda_model

aapl_lda = get_lda_model(aapl_cleaned, 10)
fb_lda = get_lda_model(fb_cleaned, 10)
pg_lda = get_lda_model(pg_cleaned, 10)#

## Test to see for APPLE 10 Topics

In [35]:
for i,topic in aapl_lda.show_topics(formatted=True, num_topics=len(aapl_lda.get_topics()), num_words=10):
    print(str(i)+": "+ topic)
    print()

0: 0.080*"stock" + 0.034*"market" + 0.024*"investor" + 0.016*"buy" + 0.016*"look" + 0.013*"time" + 0.013*"good" + 0.013*"term" + 0.013*"investment" + 0.012*"see"

1: 0.038*"service" + 0.024*"business" + 0.016*"music" + 0.015*"also" + 0.013*"grow" + 0.013*"position" + 0.012*"retail" + 0.012*"pay" + 0.012*"store" + 0.011*"acquisition"

2: 0.009*"government" + 0.009*"take" + 0.009*"case" + 0.009*"car" + 0.009*"work" + 0.008*"tech" + 0.008*"income" + 0.008*"business" + 0.007*"country" + 0.007*"tax"

3: 0.018*"user" + 0.015*"datum" + 0.014*"technology" + 0.014*"even" + 0.014*"call" + 0.011*"cloud" + 0.010*"platform" + 0.010*"internet" + 0.009*"information" + 0.009*"provide"

4: 0.051*"earning" + 0.042*"quarter" + 0.031*"revenue" + 0.028*"report" + 0.028*"share" + 0.027*"growth" + 0.025*"expect" + 0.021*"estimate" + 0.017*"sale" + 0.016*"stock"

5: 0.029*"high" + 0.024*"week" + 0.020*"market" + 0.020*"stock" + 0.019*"day" + 0.016*"low" + 0.016*"trade" + 0.015*"index" + 0.014*"close" + 0.013*

# Get Probability Distribution

In [44]:
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Darius\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
def get_topic_list(lda_model, text_cleaned, text):
  dictionary_LDA = corpora.Dictionary(text_cleaned)
  dictionary_LDA.filter_extremes(no_below=3)
  topic_lst = []
  for i in range(len(text)):
      new_text = text[i]
      tokens = [PorterStemmer().stem(token) for token in word_tokenize(new_text.lower()) if token not in stop_words]
      topic_prob_lst = lda_model[dictionary_LDA.doc2bow(tokens)]
      topic_lst.append(topic_prob_lst)
  return topic_lst

aapl_topics = get_topic_list(aapl_lda, aapl_cleaned, aapl_text)
fb_topics = get_topic_list(fb_lda, fb_cleaned, fb_text)
pg_topics = get_topic_list(pg_lda, pg_cleaned, pg_text)

In [46]:
aapl_data[[i for i in range(10)]] = 0.0
fb_data[[i for i in range(10)]] = 0.0
pg_data[[i for i in range(10)]] = 0.0
aapl_data.head()

,Date,Text,neg,neu,pos,compound,probability,sentiment,0,1,2,3,4,5,6,7,8,9
0,2021-11-10,Apple announced Wednesday it's launching a bus...,0.000,1.000,0.000,0.0000,0.621849,POSITIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-11-10,Apple will be forced to allow app developers t...,0.161,0.803,0.036,-0.7579,0.999783,NEGATIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-11-10,German chipmaker and Apple-supplier Infineon i...,0.070,0.930,0.000,-0.2846,0.999716,NEGATIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-11-10,Tim Cook's company may have made your phone or...,0.129,0.871,0.000,-0.8047,0.998617,NEGATIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-11-10,Here are the biggest calls on Wall Street on W...,0.000,0.918,0.082,0.6369,0.999696,NEGATIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
def get_prob_distr(data, topic_lst):
  for i in range(len(topic_lst)):
    prob_distr = topic_lst[i]
    for topic_num, p in prob_distr:
        #print(topic_num)
        #print(type(data.loc[i, topic_num]))
        data.loc[i, topic_num] = p
  return data

aapl_data = get_prob_distr(aapl_data,aapl_topics)
fb_data = get_prob_distr(fb_data,fb_topics)
pg_data = get_prob_distr(pg_data,pg_topics)

In [48]:
aapl_data.to_csv("Apple Topics(Ten).csv")

In [49]:
fb_data.to_csv("Facebook Final.csv")
pg_data.to_csv("P&G Final.csv")

In [52]:
aapl_data.head()

,Date,Text,neg,neu,pos,compound,probability,sentiment,0,1,2,3,4,5,6,7,8,9
0,2021-11-10,Apple announced Wednesday it's launching a bus...,0.000,1.000,0.000,0.0000,0.621849,POSITIVE,0.000000,0.000000,0.115491,0.000000,0.082169,0.0,0.527206,0.085036,0.000000,0.186787
1,2021-11-10,Apple will be forced to allow app developers t...,0.161,0.803,0.036,-0.7579,0.999783,NEGATIVE,0.000000,0.000000,0.229907,0.000000,0.000000,0.0,0.662328,0.000000,0.102421,0.000000
2,2021-11-10,German chipmaker and Apple-supplier Infineon i...,0.070,0.930,0.000,-0.2846,0.999716,NEGATIVE,0.000000,0.251074,0.211776,0.000000,0.210056,0.0,0.092855,0.230693,0.000000,0.000000
3,2021-11-10,Tim Cook's company may have made your phone or...,0.129,0.871,0.000,-0.8047,0.998617,NEGATIVE,0.060637,0.058597,0.057291,0.066928,0.000000,0.0,0.000000,0.000000,0.000000,0.753930
4,2021-11-10,Here are the biggest calls on Wall Street on W...,0.000,0.918,0.082,0.6369,0.999696,NEGATIVE,0.000000,0.000000,0.300530,0.066022,0.000000,0.0,0.055156,0.114398,0.103100,0.358984


### Combine google trend data(only for APPLE)

In [53]:
trend = pd.read_csv('apple_trend.csv')
apple_trend['Date'] = list(map(lambda x: str(x.date()), list(apple_trend.index)))
aapl_final = aapl_data.merge(apple_trend, on = 'Date', how = 'left')
aapl_final.to_csv('Apple Final.csv')